In [2]:
# -*- coding: utf-8 -*-
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import ht
import CoolProp as cp
import fluids

#### EJERCICIO PRACTICO
Un intercambiador de calor de casco y tubos (condensador), se utiliza para condensar vapor saturado de amoníaco que ingresa por la carcasa a 50°C. El fluido de enfriamiento es agua que ingresa por los tubos a una temperatura de 20°C. La transferencia de calor es de 200kW. Considerando un coeficiente global de transmisión de 1000W/m$^2$ K. Se pide determinar:

    1) La masa de vapor condensado, si el calor latente de cambio de estado es de 1051kJ/kg.
    2) El área para lograr una efectividad del intercambiador de calor del 60 por ciento con una temperatura del agua de salida de 40°C.
    3) ¿Qué porcentaje de reducción en la transferencia de calor resultaría si el flujo de agua se reduce a la mitad mientras se mantiene el área del intercambiador de calor y U iguales?.

In [3]:
fluid1,fluid2 = ['Ammonia','Water']
T1_in = 50
T2_in = 20
g = 9.8
Q = 200e3
U = 1000
Tw = (T1_in+T2_in)/2
L  = 1
D = 25.4e-3
Ntubos = 30

Area_tubo = L*D*np.pi
Area_total = Area_tubo*Ntubos
Psat = cp.CoolProp.PropsSI('P','T',50+273,'Q',1,fluid1)
rho_1v = cp.CoolProp.PropsSI('D','T',50+273,'Q',1,fluid1)
rho_1l = cp.CoolProp.PropsSI('D','T',50+273,'Q',0,fluid1)
k_1l = cp.CoolProp.PropsSI('L','T',50+273,'Q',0,fluid1)
mu_1l = cp.CoolProp.PropsSI('V','T',50+273,'Q',0,fluid1)
cp_1 = cp.CoolProp.PropsSI('C','T',50+273,'Q',1,fluid1)

h_1_v = cp.CoolProp.PropsSI('H','T',50+273,'Q',1,fluid1)
h_1_l = cp.CoolProp.PropsSI('H','T',50+273,'Q',0,fluid1)
hfg_1 = (h_1_v-h_1_l)
m_1 = Q / hfg_1
print('masa de condensado: %.2f kg/s'%(m_1))

masa de condensado: 0.19 kg/s


In [4]:
Area_total

2.3938936020354222

In [9]:
h_D


6365.604896629942

In [25]:
hcond = ht.condensation.Nusselt_laminar(T1_in,Tw,rho_1v,rho_1l,k_1l,mu_1l,hfg_1,D)

Qcond  = hcond*Area_total*(T1_in-Tw)

In [26]:
#NuD = 0.728*(rho_1l*(rho_1l-rho_1v)*ge*hfg_1*D**3/(mu_1l*k_1l*(T1_in-Tw)))**(1/4)

h_D = 0.729*(rho_1l*(rho_1l-rho_1v)*g*hfg_1*k_1l**3/(mu_1l*D*(T1_in-Tw)))**(1/4)
Qcond2  = h_D*Area_total*(T1_in-Tw)
Qcond,Qcond2

(295668.8980121843, 228578.7125269166)

In [41]:
T2_out = 40
eps = 0.6
T2_m = (T2_out+T2_in)/2
DeltaT_2 = (T2_out-T2_in)
rho_2 = cp.CoolProp.PropsSI('D','T',T2_m+273,'Q',1,fluid2)
cp_2 = cp.CoolProp.PropsSI('C','T',T2_m+273,'Q',1,fluid2)

m_2 = Q / (cp_2*(DeltaT_2))
Cmin = m_2*cp_2

In [42]:
Cmin

10000.0

In [43]:
Qmax = Q/eps

Cmin  = Qmax/(T1_in-T2_in)


In [44]:
Cmin

11111.111111111113

In [45]:
NUT = ht.NTU_from_effectiveness(eps, 0, subtype='counterflow')

In [46]:
NUT = ht.NTU_from_effectiveness(eps, 0, subtype='S&T')
UA = NUT*Cmin
A = UA/U
print('Area: %.2f m2'%A)

Area: 10.18 m2


In [40]:
NUT,eps

(1.0986122866681098, 0.666666666)

In [32]:
m_2 = m_2/2
Cmin = m_2*cp_2
Qmax = Cmin*(T1_in-T2_in)
NUT = UA/Cmin
eps2 = ht.effectiveness_from_NTU(NUT, 0, subtype='S&T')
Q2 = eps2*Qmax
print('Calor intercambiado: %.1f kW'%(Q2*1e-3))

Calor intercambiado: 133.3 kW


In [22]:
Qmax

150000.0

In [24]:
eps2

0.8888888886885806